2. Select relevant attributes from the IMDb dataset and reload.

In [20]:
import pandas as pd 
import numpy as np 
import time

In [4]:
name_basics = pd.read_csv('data/name.basics.tsv', sep = '\t', usecols=['nconst', 'primaryName', 'primaryProfession', 'knownForTitles'])

In [14]:
primary_profession = name_basics['primaryProfession'].unique()
print("primary_profession types:", primary_profession)

primary_profession types: ['actor,miscellaneous,producer' 'actress,soundtrack,archive_footage'
 'actress,music_department,producer' ... 'stunts,publicist,executive'
 'costume_department,camera_department,location_management'
 'composer,director,archive_footage']


In [15]:
unique_primary_professions = name_basics['primaryProfession'].str.split(',').explode().unique()
print("Unique primary professions:", unique_primary_professions)

Unique primary professions: ['actor' 'miscellaneous' 'producer' 'actress' 'soundtrack'
 'archive_footage' 'music_department' 'writer' 'director' 'stunts'
 'make_up_department' 'composer' 'assistant_director' 'camera_department'
 'music_artist' 'art_department' 'editor' 'cinematographer'
 'casting_director' 'executive' 'costume_designer' 'script_department'
 'art_director' 'editorial_department' 'costume_department'
 'animation_department' 'talent_agent' 'archive_sound'
 'production_designer' 'special_effects' 'production_manager'
 'sound_department' 'casting_department' 'visual_effects'
 'location_management' 'set_decorator' 'transportation_department' '\\N'
 'choreographer' 'legal' 'manager' 'publicist' 'podcaster' 'assistant'
 'production_department' 'accountant' 'electrical_department']


In [5]:
title_basics = pd.read_csv('data/title.basics.tsv', sep = '\t', usecols=['tconst', 'titleType', 'primaryTitle', 'genres'])

In [6]:
title_types = title_basics['titleType'].unique()
print("titleType categories:", title_types)

genres = title_basics['genres'].unique()
print("Genres categories:", genres)

titleType categories: ['short' 'movie' 'tvShort' 'tvMovie' 'tvEpisode' 'tvSeries' 'tvMiniSeries'
 'tvSpecial' 'video' 'videoGame' 'tvPilot']
Genres categories: ['Documentary,Short' 'Animation,Short' 'Animation,Comedy,Romance' ...
 'Biography,Crime,Fantasy' 'Mystery,Reality-TV,Thriller'
 'Musical,Reality-TV,Talk-Show']


My quiz is focus only in titleType category: 'movie'

In [ ]:
movies = title_basics[title_basics['titleType'] == 'movie']
unique_genres_movies = movies['genres'].str.split(',').explode().dropna().unique()
print("Unique genres movies:", unique_genres_movies)

Unique genres movies: ['Romance' 'Documentary' 'News' 'Sport' '\\N' 'Action' 'Adventure'
 'Biography' 'Drama' 'Fantasy' 'Comedy' 'War' 'Crime' 'Family' 'History'
 'Sci-Fi' 'Thriller' 'Western' 'Mystery' 'Horror' 'Music' 'Animation'
 'Musical' 'Film-Noir' 'Talk-Show' 'Adult' 'Reality-TV' 'Game-Show']


My quiz is not focus on the following unique genres categories: 'News', '\\N', 'Animation', 'Film-Noir, 'Talk-Show, 'Adult', 'Reality-TV', 'Game-Show'

In [30]:
filtered_titles = title_basics[
    (title_basics['titleType'] == 'movie') & 
    (~title_basics['genres'].str.contains(r'Animation|News|\\N|Film-Noir|Talk-Show|Adult|Reality-TV|Game-Show', na=False, regex=True))
]

In [10]:
title_crew = pd.read_csv('data/title.crew.tsv', sep = '\t', usecols=['tconst', 'directors'])

In [11]:
title_ratings = pd.read_csv('data/title.ratings.tsv', sep = '\t')

In [12]:
title_principals = pd.read_csv('data/title.principals.tsv', sep = '\t', usecols=['tconst', 'ordering', 'nconst', 'category'])

In [19]:
num_rows = title_principals.shape[0]
num_columns = title_principals.shape[1]
print(num_rows)
print(num_columns)

88962941
4


In [16]:
categories = title_principals['category'].unique()
print(categories)

['self' 'director' 'producer' 'cinematographer' 'composer' 'editor'
 'actor' 'actress' 'writer' 'production_designer' 'archive_footage'
 'casting_director' 'archive_sound']


In [24]:

chunk_sizes = [500000, 5000, 200000]

for chunk_size in chunk_sizes:
    start_time = time.time()
    for chunk in pd.read_csv('data/title.principals.tsv', sep='\t', chunksize=chunk_size):
        pass 
    end_time = time.time()
    print(f"Chunk_size time={chunk_size}: {end_time - start_time:.2f} seconds")


Chunk_size time=500000: 152.55 seconds
Chunk_size time=5000: 166.42 seconds
Chunk_size time=200000: 86.73 seconds


In [28]:
filtered_title_principals = title_principals[title_principals['category'].isin(['actor', 'actress', 'director'])]